In [1]:
import requests 
import pandas as pd
import json
import pickle
import nltk
import string

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from keras.models import load_model
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

In [85]:
train = pd.read_csv('train.csv')
train.dropna(inplace = True)
X = train['TEXT']
y = train['truth']

vectorizer = TfidfVectorizer()
vectors_train = vectorizer.fit_transform(X)

tokenizer = Tokenizer(lower=True, char_level=False)
tokenizer.fit_on_texts(train["TEXT"].tolist())

rfc = pickle.load(open('RandomForest.sav', 'rb'))
gbc = pickle.load(open('Gradient-Boost.sav', 'rb'))
etc = pickle.load(open('Extra-Trees.sav', 'rb'))
bnb = pickle.load(open('BernoulliNB.sav', 'rb'))
cnn = load_model('CNN.h5')

In [86]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

tokenicer = RegexpTokenizer(r'\w+')

def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = " ".join([lemmatizer.lemmatize(i) for i in text])
    return lem_text

In [87]:
teamToken = "zJNrIv24ExPOrOmGw65tG3Ht6rCY4wfbifE6hcVU87U"
get = 'https://erisk.irlab.org/challenge-t1/getwritings/'+teamToken
post= 'https://erisk.irlab.org/challenge-t1/submit/'+teamToken+'/'

In [ ]:
main = []

nick = "nick"
dec = "decision"
sc = "score"
count = 1

r = requests.get(url = get)
data = r.json()
for d in data:
    main.append([d['content'], d['nick']])
maindf = pd.DataFrame(main, columns = ['content', 'nick'])

maindf['content'] = maindf['content'].apply(lambda x: remove_punctuation(x), 1)
maindf['content'] = maindf['content'].apply(lambda x: tokenicer.tokenize(x.lower()))
maindf['content'] = maindf['content'].apply(lambda x: remove_stopwords(x))
maindf['content'] = maindf['content'].apply(lambda x: word_lemmatizer(x))
maindf['content'] = maindf['content'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
maindf['content'] = maindf['content'].apply(lambda elem: re.sub(r"\d+", "", elem))

print('Got Sequence 35', "\n")

print("\nTotal Number of Users: 423")
    
while(len(data) != 0):
    vectors = vectorizer.transform(maindf['content'])
    tempcnn = tokenizer.texts_to_sequences(maindf['content'])    
    tempcnn = pad_sequences(tempcnn, maxlen=200)    
    for i in range(5):
        send = []
        if(i == 0):
            print('\nProcessing Run Bernoulli')
            decision = bnb.predict(vectors)
            prob = bnb.predict_proba(vectors)
            score = [max(p) for p in prob]
        elif(i == 1):    
            print('Processing Run Random Forest')
            decision = rfc.predict(vectors)
            prob = rfc.predict_proba(vectors)
            score = [max(p) for p in prob]
        elif(i == 2):  
            print('Processing Run Gradient Boosting')
            decision = gbc.predict(vectors)
            prob = gbc.predict_proba(vectors)
            score = [max(p) for p in prob]
        elif(i == 3):    
            print('Processing Run CNN')
            predictions = cnn.predict(tempcnn, batch_size=1024, verbose=1)
            y_pred = np.argmax(predictions, axis=1)
            decision = [1 if x == 0 else 0 for x in y_pred]
            score = [max(p) for p in predictions]
        elif(i == 4):    
            print('Processing Run Extra Trees')
            decision = etc.predict(vectors)
            prob = etc.predict_proba(vectors)
            score = [max(p) for p in prob]

        for k in range(len(maindf)):
                send.append({nick:str(maindf['nick'][k]),
                              dec:int(decision[k]),
                              sc:float(score[k])})
        
        requests.post(url = post+str(i), json = send)
        
    r = requests.get(url = get)
    
    count += 1
    print('\nGot Sequence ', count)
    
    data = r.json()
    
    templ = []
    tempdf = pd.DataFrame()

    for d in data:
           templ.append([d['content'], d['nick']])
    tempdf = pd.DataFrame(templ, columns = ['content', 'nick'])
    
    tempdf['content'] = tempdf['content'].apply(lambda x: remove_punctuation(x), 1)
    tempdf['content'] = tempdf['content'].apply(lambda x: tokenicer.tokenize(x.lower()))
    tempdf['content'] = tempdf['content'].apply(lambda x: remove_stopwords(x))
    tempdf['content'] = tempdf['content'].apply(lambda x: word_lemmatizer(x))
    tempdf['content'] = tempdf['content'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    tempdf['content'] = tempdf['content'].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    print("\nNumber of users data retrieved:", len(data))
       
    for i1, r1 in maindf.iterrows():
        for i2, r2 in tempdf.iterrows():
            if(r1['nick'] == r2['nick']):
                r1['content'] = r1['content'] + r2['content']

Got Sequence 35 


Total Number of Users: 423

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 129us/step
Processing Run Extra Trees

Got Sequence  2

Number of users data retrieved: 309

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 225us/step
Processing Run Extra Trees

Got Sequence  3

Number of users data retrieved: 309

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 130us/step
Processing Run Extra Trees

Got Sequence  4

Number of users data retrieved: 305

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 129us/step
Processing Run Extra Trees

Got Sequence  5

Number of 

Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 247us/step
Processing Run Extra Trees

Got Sequence  35

Number of users data retrieved: 243

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 239us/step
Processing Run Extra Trees

Got Sequence  36

Number of users data retrieved: 241

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 279us/step
Processing Run Extra Trees

Got Sequence  37

Number of users data retrieved: 240

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 356us/step
Processing Run Extra Trees

Got Sequence  38

Number of users data retrieved: 240

Processing Run Bernoulli
Processing Run Random Forest
Processing Run G

Processing Run Extra Trees

Got Sequence  68

Number of users data retrieved: 203

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 333us/step
Processing Run Extra Trees

Got Sequence  69

Number of users data retrieved: 203

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 373us/step
Processing Run Extra Trees

Got Sequence  70

Number of users data retrieved: 202

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 342us/step
Processing Run Extra Trees

Got Sequence  71

Number of users data retrieved: 202

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 317us/step
Processing Run E


Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 204us/step
Processing Run Extra Trees

Got Sequence  102

Number of users data retrieved: 182

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 339us/step
Processing Run Extra Trees

Got Sequence  103

Number of users data retrieved: 182

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 334us/step
Processing Run Extra Trees

Got Sequence  104

Number of users data retrieved: 182

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 295us/step
Processing Run Extra Trees

Got Sequence  105

Number of users data retrieved: 182

Processing 

Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 228us/step
Processing Run Extra Trees

Got Sequence  135

Number of users data retrieved: 164

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 215us/step
Processing Run Extra Trees

Got Sequence  136

Number of users data retrieved: 163

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 224us/step
Processing Run Extra Trees

Got Sequence  137

Number of users data retrieved: 161

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 183us/step
Processing Run Extra Trees

Got Sequence  138

Number of users data retrieved: 160

Processing Run Bernoulli
Processing Run Random Forest
Processing R

...

Processing Run CNN
423/423 [==============================] - 0s 176us/step
Processing Run Extra Trees

Got Sequence  435

Number of users data retrieved: 71

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 351us/step
Processing Run Extra Trees

Got Sequence  436

Number of users data retrieved: 71

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 225us/step
Processing Run Extra Trees

Got Sequence  437

Number of users data retrieved: 71

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 239us/step
Processing Run Extra Trees

Got Sequence  438

Number of users data retrieved: 71

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run C

Processing Run Extra Trees

Got Sequence  468

Number of users data retrieved: 65

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 160us/step
Processing Run Extra Trees

Got Sequence  469

Number of users data retrieved: 65

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 219us/step
Processing Run Extra Trees

Got Sequence  470

Number of users data retrieved: 65

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 387us/step
Processing Run Extra Trees

Got Sequence  471

Number of users data retrieved: 65

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 313us/step
Processing Run E


Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 158us/step
Processing Run Extra Trees

Got Sequence  502

Number of users data retrieved: 59

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 229us/step
Processing Run Extra Trees

Got Sequence  503

Number of users data retrieved: 59

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 225us/step
Processing Run Extra Trees

Got Sequence  504

Number of users data retrieved: 59

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 156us/step
Processing Run Extra Trees

Got Sequence  505

Number of users data retrieved: 59

Processing Run 

Processing Run CNN
423/423 [==============================] - 0s 219us/step
Processing Run Extra Trees

Got Sequence  535

Number of users data retrieved: 56

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 136us/step
Processing Run Extra Trees

Got Sequence  536

Number of users data retrieved: 56

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 213us/step
Processing Run Extra Trees

Got Sequence  537

Number of users data retrieved: 56

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 216us/step
Processing Run Extra Trees

Got Sequence  538

Number of users data retrieved: 55

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run C

Processing Run Extra Trees

Got Sequence  568

Number of users data retrieved: 52

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 140us/step
Processing Run Extra Trees

Got Sequence  569

Number of users data retrieved: 52

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 170us/step
Processing Run Extra Trees

Got Sequence  570

Number of users data retrieved: 52

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 156us/step
Processing Run Extra Trees

Got Sequence  571

Number of users data retrieved: 52

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 154us/step
Processing Run E